## Notebook для конкурса

Что тут происходит:

Сначала из каждого изображения удаляются шумы -- ненулевые пиксели, вокруг которых есть хотя бы 5 нулевых (вокруг -- это -1, 0, +1 по оси х и также по у, удаление шумов делает измерение расстояния между изображениями более точным), а также пиксели, которые темнее, чем 100, обнуляются (это могло помочь там, где изображение состоит из двух изображений цифр, наложенных друг на друга с некоторыми коэффициентами, при обнулении пикселей, которые темнее 100, там во многих случаях остается только одно). Это все делается в функции exclude_noices. Затем на обработанной тренировочной выборке обучается KNeighborsClassifier с параметрами k=3 и weights='distance' и делается predict на тестовой (каждое изображение из тестовой выборки тоже обрабатывается функцией exclude_noices).

Вот. Все просто.

(Лучшая посылка на kaggle была получена именно так, вторая посылка, отобранная на прогон по всей тестовой выборке, 
была получена тем же алгоритмом, но без обнуления пикселей, которые темнее, чем 100)

In [1]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from matplotlib import pyplot

%load_ext autoreload
%autoreload 2

 Считываем данные:

In [2]:
X_train = pd.read_csv('hard_train.txt', header=None, sep=' ').values
Y_train = pd.read_csv('hard_train_labels.txt', header=None, sep=' ').values
X_test = pd.read_csv('hard_test.txt', header=None, sep=' ').values

Функция, исключающая "шумы" и обнуляющая неяркие пиксели:

(А, ну еще пиксели на границе -- где х < 2 или х > 25 или у < 2 или у > 25, делаем все нулевыми)

In [3]:
def exclude_noices(picture):
    p = picture
    
    for i in range(28):
        for j in range(28):
            if (i<2 or i>25 or j<2 or j>25):
                picture[i][j] = 0
                continue
            if picture[i][j] < 100:
                picture[i][j] = 0
            sum = 0
            for a in range(-1, 2):
                for b in range(-1, 2):
                    if p[i + a][j + b] == 0:
                        sum += 1
            if (sum > 5):
                picture[i][j] = 0
    return picture

Применяем к выборкам функцию exclude_noises:

reshape, чтобы легче было обрабатывать 

In [ ]:
X_train_new = [[] for i in range(200000)]
for i in range(200000):
    picture = X_train[[i]].reshape(28, 28)
    X_train_new[i] = exclude_noices(picture)
    
X_test_new = [[] for i in range(20000)]
for i in range(20000):
    picture = X_test[[i]].reshape(28, 28)
    X_test_new[i] = exclude_noices(picture)

reshape обратно:

In [5]:
for i in range(200000):
    X_train_new[i] = X_train_new[i].reshape((784))

Обучаем kNN:

In [8]:
knn = KNeighborsClassifier(n_neighbors=3, weights='distance')
knn.fit(X_train_new, np.ravel(Y_train))        

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='distance')

reshape тестовой выборки обратно:

In [9]:
for i in range(20000):
    X_test_new[i] = X_test_new[i].reshape((784))

собственно predict:

In [11]:
% time y_predicted = knn.predict(X_test_new)

CPU times: user 1h 35min 17s, sys: 712 ms, total: 1h 35min 18s
Wall time: 1h 35min 15s


Сохраняем в файл

In [12]:
pd.DataFrame(y_predicted).to_csv('y_predicted_k3_exn_3.csv')

Все, win